--- Daniel Maidment ---
$\renewcommand{\vec}{\mathbf}$
$\newcommand{\x}{\vec{x}}$
$\newcommand{\X}{\vec{X}}$
$\newcommand{\s}{\vec{s}}$
$\renewcommand{\phi}{\varphi}$
$\newcommand{\R}{\mathbb{R}}$
$\newcommand{\y}{\vec{y}}$
$\renewcommand{\v}{\vec{v}}$
$\newcommand{\A}{\vec{A}}$
$\newcommand{\I}{\vec{I}}$
$\newcommand{\z}{\vec{z}}$
$\newcommand{\a}{\vec{a}}$
$\newcommand{\b}{\vec{b}}$
$\newcommand{\t}{\vec{t}}$
$\newcommand{\W}{\vec{W}}$
$\newcommand{\w}{\;\!}$
$\newcommand{\l}{\w\left(l\right)}$
$\newcommand{\th}{\text{th}}$
$\newcommand{\(}{\left (}$
$\newcommand{\)}{\right )}$

# Architecture of the binary spiking neural network

## Neuron

### Hidden layer neuron

The fundamental neuron design encompasses the idea that the neuron should model the integrate and fire mode of organic neurons.

![Linear hidden layer neuron](images/Diagrams/Neurons/Accumulator_Neuron_General.png)\label{fig:linear_neuron}
##### Linear hidden layer neuron

Here the hidden layer neurons are implemented as binary accumulators. Each neuron takes the summed synaptic input. If more than one of the input synapses fire simultaneously, the event is treated as a single firing event and the neuron only takes one input. This is for one major reason, in that the summation is a logical OR, hence if a synapse takes a single input or many inputs, the summation results in a True output. A True output from the OR gate results in an aggregation of ones in the neuron's register. After N ones have been accumulated the neuron fires a single 1. Subsequently, the neuron is reset. This implies that the minimum firing time is N clock cycles. Hence, the length of the neuron's register modulates the maximum firing rate. Modulating the size of the value of N, or altering how the register is filled upon reset, therefore, can modulate the firing rate. Hence the neuron's length is a meta-parameter of the network.

<img src="images/neuron/linear_neuron_frequency_model_11_10000.png" alt="Frequency model of the linear hidden layer neuron."
	title="Frequency model of the linear hidden layer neuron." width="400" height="400" />\label{fig:linear_neuron_frequency_model_11_10000}
##### Frequency model of the linear hidden layer neuron.

### Hidden layer neuron with feedback

![Hidden layer neuron with feedback](images/Diagrams/Neurons/feedback_neuron.png)\label{fig:feedback_neuron}
##### Hidden layer neuron with feedback

<img src="images/neuron/delayFB2/feedback_neuron_frequency_model_smallNFF_smallNFB.png" alt="Frequency model of the hidden layer neuron with feedback for small values of NFF where NFB=NFF-1."
	title="Frequency model of the hidden layer neuron with feedback for small values of NFF where NFB=NFF-1." width="400" height="400" />\label{fig:feedback_neuron_frequency_model_smallNFF_smallNFB}
##### Frequency model of the hidden layer neuron with feedback for small values of NFF where NFB=NFF-1.

<img src="images/neuron/delayFB2/frequency_model_NFF4_smallNFB.png" alt="Frequency model of the hidden layer neuron with feedback where of NFF=4 and NFB is small."
	title="Frequency model of the hidden layer neuron with feedback where of NFF=4 and NFB is small." width="400" height="400" />\label{fig:frequency_model_NFF4_smallNFB}
##### Frequency model of the hidden layer neuron with feedback where of NFF=4 and NFB is small.

### Input neuron

![Hidden layer neuron with feedback](images/Diagrams/Neurons/Accumulator_Neuron_Input.png)\label{fig:feedback_neuron}
##### Input neuron

The input neuron is fundamentally different from the remaining neurons in the network. The premise of the binary spiking neural network is that it reduces the complexity of the basic operations of a standard feed-forward neural network. The matrix multiplications of the standard FFNN are reduced to simultaneous binary AND operations across the network that execute in a single clock cycle. However, this construction requires a conversion of real world input parameters (typically some decimal value) into binary. The chosen method of conversion requires normalising the input value to some $p$ where
\begin{equation}
    p \in (0, 1).
\end{equation}
The normalised value is then used to seed the a binary number generator where:
\begin{equation}
    P(x) = 
    \begin{cases}
        p, &x=1,
        \\
        1-p, &x=0.
    \end{cases}
\end{equation}
This operation for a given input value occurs for some number of clock cycles N. Hence N is the time framing for a given input. It follows that for some number of network layers L, the minimum time required for the network to process some input is approximately $\frac{N+L}{f_{\text{ck}}}$, where $\frac{1}{f_{\text{ck}}}$ is the period of a the system clock. It also follows that if the output $x$ is an independent sample of a Bernoulli process with probability $p$ then
\begin{equation}
    E(x) \to p \text{ as } N \to \infty.
\end{equation}

## Synapse

![Hidden layer neuron with feedback](images/Diagrams/Synapse/cyclic_Synapse.png)\label{fig:feedback_neuron}
##### Binary cyclic synapse

Synapses are implemented as cyclic shift registers, where the output of the shift register feeds the input. The output of the shift register acts as the first input to a two-input AND gate. The second input is the output of the presynaptic neuron. The output of the AND gate is an input to a postsynaptic neuron. The function of the synapse is to regulate the firing rate of the presynaptic neuron, and the accumulation rate -and hence the firing rate- of the post-synaptic neuron. 

Each element of the shift register is seeded with a sample from a Bernoulli process value that has some probability $p_{\text{synapse}}$ of being a 1, and is otherwise a 0. This is given by:
\begin{equation}
    P(x) = 
    \begin{cases}
        p, &x=1,
        \\
        1-p, &x=0.
    \end{cases}
\end{equation}

In the case of a synapse of length N, the expected value of the shift register is:
\begin{equation}
    E(x)=\frac{\sum_{i=0}^{N-1} x_i}{N},
\end{equation}
where $x_i$ is the value of an element in the shift register. Hence,
\begin{equation}
    E(x) \to p_{\text{synapse}} \text{ as } N \to \infty.
\end{equation}

By ANDing the output of the register with the output of the presynaptic neuron, we are effectively multiplying the expected value of the register over time with the firing rate of the presynaptic neuron ($f_{\text{pre}}$). Hence, the apparent firing rate of the synapse becomes:
\begin{aligned}
    f_{\text{synapse}} &= E(x)\cdot f_{\text{pre}}\\
                       &= \frac{\sum_{i=0}^{N-1} x_i}{N} \cdot f_{\text{pre}}.
\end{aligned}
Then as $N\to \infty$:
\begin{equation}
    f_{\text{synapse}} = p_{\text{synapse}}\cdot f_{\text{pre}}. \label{eq:f_ideal}
\end{equation}
This is important because by seeding a synapse with some Bernoulli process where $P(x=1)=p_{\text{synapse}}$ we can define the weight of the synapse $w_{i\w j}\approx p_{\text{synapse}}$, which gives us some means of translating an ordinary weight from a FFNN into some frequency modulating weight.

<img src="images/synapse/Synapse_frequency_model_N100.png" alt="Changing the synaptic seed frequency [$p_{seed}$] and input frequency for a fixed synaptic length [N]."
	title="Changing the synaptic seed frequency [$p_{seed}$] and input frequency for a fixed synaptic length [N]." width="400" height="400" />\label{fig:Synapse_frequency_model_N100}
##### Changing the synaptic seed frequency [$p_{seed}$] and input frequency for a fixed synaptic length [N].

<img src="images/synapse/Synapse_frequency_model_pseed[0.3, 0.7]_seed101.png"
width="900" height="400" />\label{fig:Synapse_frequency_model_pseed}
##### Changing the input frequency and synaptic length [N] for a fixed synaptic frequency [$p_{seed}=\{0.3, 0.7\}$].